# **Music Genre Classifier**

---





# Installing required libraries

In [1]:
!pip install scipy
!pip install python_speech_features

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for python-speech-features: filename=python_speech_features-0.6-py3-none-any.whl size=5889 sha256=a5d6e928753be540d8c6221db047031e46d1e07a0de5cc46c323eda13341a787
  Stored in directory: /root/.cache/pip/wheels/b0/0e/94/28cd6afa3cd5998a63eef99fe31777acd7d758f59cf24839eb
Successfully built python-speech-features


# Importing libraries

In [2]:
import pandas as pd
import numpy as np
import scipy.io.wavfile as wav
from tempfile import TemporaryFile
import librosa as lib
import os as sys
import math
import random
import operator
import pickle as p


# Downloading GZTAN Genre Collection Dataset from Kaggle

In [3]:
! pip install -q kaggle
from google.colab import files
sys.mkdir('/root/.kaggle')

In [4]:
files.upload()
#copy
!cp kaggle.json /root/.kaggle/

Saving kaggle.json to kaggle.json


In [5]:
# change the permissions of the file
!chmod 600 /root/.kaggle/kaggle.json

In [6]:
# download the dataset for a specific competition
!kaggle datasets download -d carlthome/gtzan-genre-collection

 99% 1.13G/1.14G [00:07<00:00, 170MB/s]
100% 1.14G/1.14G [00:07<00:00, 156MB/s]


# Extracting features from the audio files in the dataset

In [ ]:
sys.mkdir('/content/Kaggle')
#Unzip the file
!unzip  /content/gtzan-genre-collection.zip -d /content/Kaggle/Dataset

In [205]:
#num mfcc features
num=1293

In [206]:
dataset=open("/content/dataset.dat","wb")
directory="/content/Kaggle/Dataset/genres"
i=0
# Navigating throught folder of each genre
out=[]
for folder in sys.listdir(directory):
  i+=1
  #10 genres
  if i==11:
    break
  for file in sys.listdir(directory+"/"+folder):
    loc=directory+"/"+folder+"/"+file
    sig, rate=lib.load(loc)
    mfcc_features = lib.feature.mfcc(sig, sr=rate)
    r=num
    while (r<len(mfcc_features.transpose())):
       vec=mfcc_features[:, (r-num):r]
       vec=vec.flatten()
       vec=np.append(vec, i)
       out.append(vec)
       r+=num
#using pickle to write features to the .dat file
p.dump(out,dataset)
dataset.close()

# Extracting feature vectors(X) and outputs(Y) from the dataset

In [207]:
dataset=open("/content/dataset.dat","rb")
convertedData=np.asarray(p.load(dataset))
X=[]
Y=[]
for item in convertedData:
    x=item[:-1]
    y=item[-1]
    X.append(x)
    Y.append(y)
dataset.close()

In [208]:
lim=len(X)-1
for c in range(0,lim):
  if c<=lim:
    temp=X[c]
  else:
    break
  #Removing data samples with insufficient amount of features
  if (len(temp)!=num):
    X.pop(c)
    Y.pop(c)
    lim=-1

# Split the dataset into two categories: 70 percent for training and 30 percent for testing

In [270]:
from sklearn.model_selection import train_test_split
x_train, x_test,y_train, y_test=train_test_split(X,Y,train_size=0.7, random_state=0)

# Fitting K-NN classifier to the training data (importing KNeighborsClassifier from the library Sklearn neighbors) 

In [329]:
#number of components
comps=20
from sklearn.neighbors import KNeighborsClassifier, NeighborhoodComponentsAnalysis
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.pipeline import make_pipeline
#Dimensionaity reduction using Neighborhood Components Analysis
nca = make_pipeline(StandardScaler(), NeighborhoodComponentsAnalysis(n_components=comps, random_state=0))
nca.fit(x_train, y_train)
x_dim_train=nca.transform(x_train)
nca.fit(x_test, y_test)
x_dim_test=nca.transform(x_test)

In [330]:
#number of neighbours
k=2
classifier= KNeighborsClassifier(n_neighbors=k, metric='minkowski', p=2)  
x_train=np.asarray(x_train)
y_train=np.asarray(y_train)
classifier.fit(x_dim_train, y_train)  

KNeighborsClassifier(n_neighbors=2)

# Predict results

In [331]:
#Checking on the training set
y_predicted=classifier.predict(x_dim_train)
res=y_predicted==y_train
accuracy=res.sum()*100/len(res)
print("Accuracy on the training dataset: "+str(accuracy)+" %")

Accuracy on the training dataset: 96.7741935483871 %


In [338]:
#Checking on the test set
model=classifier.fit(x_dim_test, y_test) 
y_predicted=classifier.predict(x_dim_test)
res=y_predicted==y_test
accuracy=res.sum()*100/len(res)
print("Testing Accuracy: "+str(accuracy)+" %")

Testing Accuracy: 71.42857142857143 %


# Pickle the model

In [339]:
#Saving the model
p.dump(model, open('/content/model.pkl', 'wb'))
#Uncomment to load the model
pickled_model = p.load(open('/content/model.pkl', 'rb'))
y=pickled_model.predict(x_dim_test)